# ConvLSTM

In [1]:
import numpy as np

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
# import keras
# from keras.models import Sequential
# from keras.layers import ConvLSTM2D, Dense, InputLayer, BatchNormalization, Permute

In [2]:
print(tf.__version__)

2.2.0-dlenv


Let's just make a model and experiment with it

In [3]:
frames = 1
channels = 1
pixels_x = 21
pixels_y = 21

In [4]:
model = tf.keras.Sequential(name='convlstm_model_0')
model.add(layers.InputLayer(input_shape=(frames, channels, pixels_x, pixels_y)))
model.add(layers.ConvLSTM2D(
    filters=channels*4, kernel_size=(5,5), padding='same', data_format='channels_first',return_sequences=True))
model.add(layers.BatchNormalization(axis=1, ))
model.add(layers.ConvLSTM2D(
    filters=channels*2, kernel_size=(3,3), padding='same', data_format='channels_first',return_sequences=True))
model.add(layers.BatchNormalization(axis=1, ))
model.add(layers.ConvLSTM2D(
    filters=channels, kernel_size=(1,1), padding='same', data_format='channels_first',return_sequences=True))
model.add(layers.BatchNormalization(axis=1, ))

model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['mean_absolute_error'])

In [5]:
model.summary()

Model: "convlstm_model_0"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 1, 4, 21, 21)      2016      
_________________________________________________________________
batch_normalization (BatchNo (None, 1, 4, 21, 21)      4         
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 1, 2, 21, 21)      440       
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 2, 21, 21)      4         
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 1, 1, 21, 21)      16        
_________________________________________________________________
batch_normalization_2 (Batch (None, 1, 1, 21, 21)      4         
Total params: 2,484
Trainable params: 2,478
Non-trainable params: 6
________________________________________________

## try training

In [6]:
import glob
import xarray as xr
def generate_arrays(img_dir, slice_size=24, vars_=['t2m']):
    """
    A generator that returns one 24-hour slice as input, and the subsequent 24-hour slice as output
    """
    # get list of netcdf files in img_dir
    netcdf_dirs = sorted(glob.glob(img_dir+"/*.nc"))
    file_index = 0
    # open first netcdf file
    ds = xr.open_dataset(netcdf_dirs[file_index])
    # select only some variables
    ds = ds[vars_]
    # counter is for hourly time slices. months with 31 days have 744 hours
    counter = 0
    while True: # generator needs to run infinitely
        
        # get input slice
        input_images = ds.isel( time=slice(counter, counter + slice_size)).to_array().values
        
        # check if we're at the end of the month
        if counter+2*slice_size > ds.sizes['time']:
            # reset slice counter, increment to next netcdf file, open it, get output images
            counter = 0
            file_index += 1
            if file_index == len(netcdf_dirs):
                file_index = 0
                
            ds = xr.open_dataset(netcdf_dirs[file_index])
            # select only some variables
            ds = ds[vars_]
            # take slice 0-24 as output-image
            output_images = ds.isel( time=slice(counter, counter + slice_size)).to_array().values
            # set counter to -slice_size to reset for input on next iteration
            counter -= slice_size
        # get output slice right after input slice
        else:
            output_images = ds.isel( time=slice(counter+slice_size, counter + 2*slice_size)).to_array().values
        
        # switch frames and channel axes
        input_images = np.moveaxis(input_images, 0, 1)
        output_images = np.moveaxis(output_images, 0, 1)
        # reshape values
        input_images = input_images.reshape(-1, frames, channels, pixels_x, pixels_y)
        output_images = output_images.reshape(-1, frames, channels, pixels_x, pixels_y)
        yield (input_images, output_images)
        counter += slice_size

In [7]:
train_file_path = "../data/train"
validate_file_path = "../data/validate"
# 3 years of training data = 
train_steps = 3 * 365 * 24 / frames
# 1 year of validation data = 
valid_steps = 1 * 365 * 24 / frames

In [ ]:
history = model.fit(
    generate_arrays(train_file_path, slice_size=frames),
    steps_per_epoch = train_steps,
    epochs = 20,
    verbose = 1,
    shuffle = False,
    initial_epoch = 0,
    validation_steps = valid_steps,
    validation_data = generate_arrays(validate_file_path, slice_size=frames),
    )

Epoch 1/20
26280/26280 [==============================] - 868s 33ms/step - loss: 49184.8281 - mean_absolute_error: 287.7737 - val_loss: 38606.3633 - val_mean_absolute_error: 287.5273
Epoch 2/20
26280/26280 [==============================] - 868s 33ms/step - loss: 43895.5586 - mean_absolute_error: 287.7822 - val_loss: 38714.2773 - val_mean_absolute_error: 287.5027
Epoch 3/20
26280/26280 [==============================] - 872s 33ms/step - loss: 43780.6094 - mean_absolute_error: 287.7748 - val_loss: 68717.8047 - val_mean_absolute_error: 287.5238
Epoch 4/20
26280/26280 [==============================] - 875s 33ms/step - loss: 23960.7598 - mean_absolute_error: 287.7737 - val_loss: 13661.2412 - val_mean_absolute_error: 287.5292
Epoch 5/20
26280/26280 [==============================] - 876s 33ms/step - loss: 13662.8223 - mean_absolute_error: 287.7738 - val_loss: 13618.2490 - val_mean_absolute_error: 287.5292
Epoch 6/20
26280/26280 [==============================] - ETA: 0s - loss: 13628.7480 

## Save model to ```../models/```

In [ ]:
import pickle
import datetime
current_datetime = datetime.datetime.now().strftime("%Y_%M_%d_%H%M")
pickle.dump( history, open( "../models/"+current_datetime+"_convlstm.pkl", "wb" ) )

In [ ]:
#rewrite model in Functional style

# main input
frame_input = layers.Input(shape=(frames, channels, pixels_x, pixels_y), name='frame_input')


# ConvLSTM block
stack = layers.ConvLSTM2D(filters=20, kernel_size=(5,5), padding='same',
                       data_format='channels_first',return_sequences=True, name='ConvLSTM_1')(frame_input)
stack = layers.BatchNormalization(axis=1, name='batchnorm_1')(stack)
stack = layers.ConvLSTM2D(filters=10, kernel_size=(5,5), padding='same',
                       data_format='channels_first',return_sequences=True, name='ConvLSTM_2')(stack)
stack = layers.BatchNormalization(axis=1, name='batchnorm_2')(stack)
stack = layers.ConvLSTM2D(filters=10, kernel_size=(1,1), padding='same',
                       data_format='channels_first',return_sequences=True, name='ConvLSTM_3')(stack)

clstm_out = layers.Flatten(name = 'ConvLSTM_out')(stack)

# auxiliary input
auxiliary_input = layers.Input(shape=(5,), name='aux_input')
dense_input = layers.concatenate([clstm_out, auxiliary_input], name='concatenate_layer')

# dense layers
dropout_seed = 42
dense1 = layers.Dense(16, name='first_dense_layer')(dense_input)
dropout1 = layers.Dropout(0.75, seed = dropout_seed, name='first_dropout_layer')(dense1)
dense2 = layers.Dense(8, name='second_dense_layer')(dropout1)
dropout2 = layers.Dropout(0.75, seed = dropout_seed, name='second_dropout_layer')(dense2)
dense3 = layers.Dense(4, name='third_dense_layer')(dropout2)
dropout3 = layers.Dropout(0.75, seed = dropout_seed, name='third_dropout_layer')(dense3)
dense3 = layers.Dense(1, name='final_dense_layer')(dropout3)

# reshape output
# dense_output = layers.Reshape(target_shape=(frames, channels, pixels_x, pixels_y))(dense3)


model2 = Model(inputs = [frame_input, auxiliary_input], outputs = [dense3])

model2.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['mean_absolute_error'])
model2.summary()